# Shor's Algorithm

In [4]:
from qiskit import QuantumCircuit, QuantumRegister

## Auxilary methods

### Reverse binary string

In [5]:
# Returns a string 's' of 'n' bit binary representation of an integer 'a' s.t. s[i] = a_i
# i.e. a mod 2^n if a > 2^n
# and 0..0a if a < 2^n
def n_bit_binary(n: int, a: int):
    tmp = bin(a)
    l = len(tmp)
    ans = ""

    d = min(l - 2, n)
    for i in range(d):
        ans += tmp[l - i - 1]

    d = n - (l - 2)
    for i in range(d):
        ans += '0'

    return ans

### Logarithm base 2

In [6]:
# Precondition: x > 0
def log_2(x: int):
    c = 1
    i = 0
    while c < x:
        c *= 2
        i += 1
    return i

### Multiplicative Inverse

In [7]:
# Returns a' s.t. a(a') = 1 mod N
# Precondition: GCD(a, N) = 1  
def multiplicative_inverse(a: int, N: int):
    for i in range(1, N):
        if (a * i) % N == 1:
            return i
    print("ERROR: Multipliciative invere of ", a, " related to ", N, " does not exist.")
    return -1

### Circuit Initialization

In [9]:
# Takes a circuit 'qc', and post-composes it with initialization of 'map' qubits with 'n' bit int 'num'
# Precondition: len(map) == n
def init_qreg_val(n: int, num: int, qc: QuantumCircuit, map):
    qc_init = QuantumCircuit(n)

    bin_num = n_bit_binary(n, num)

    for i in range(n):
        if bin_num[i] == '1':
            qc_init.x(i)
    
    return qc.compose(qc_init, qubits=map)


def _test_init_qreg_val():
    n = 6
    A = QuantumRegister(n, name='A')
    B = QuantumRegister(n, name='B')
    qc = QuantumCircuit(A, B)

    qc = init_qreg_val(n, 6, qc, map=[2 * i for i in range(n)]) # 6 = 110
    qc.draw(output='mpl')

#### Uniform Superposition

In [10]:
def uniform_superposiotion_creator(n: int):
    qc = QuantumCircuit(n)

    for i in range(n):
        qc.h(i)

    return qc


def _test_uniform_superposiotion_creator():
    qc = uniform_superposiotion_creator(n=5)
    print(qc)

### Testing

#### Measurement

In [11]:
from qiskit.quantum_info import Pauli

# Returns Pauli operator for measuring q_th qubit from n qubits (counting from 0) in Z basis
# Precondition: q < n
def measure_z(n: int, q: int):
    p = ''
    for i in range(n):
        if i == q:
            p += 'Z'
        else:
            p += 'I'
    return p


# Returns Observables based on Pauli operators for measuring all 'n' qubits in Z basis
def observables(n: int):
    obs = []
    for i in range(n-1, -1, -1):
        obs.append(measure_z(n, i))
    return obs

In [13]:
from qiskit_aer.primitives import Estimator

def test_all(qc: QuantumCircuit, is_exact: bool):
    n = len(qc.qubits)
    obs = observables(n)
    return estimate(qc, obs, is_exact)


def estimate(qc: QuantumCircuit, observables, is_exact: bool):
    estimator = Estimator()

    job = estimator.run([qc] * len(observables), observables)

    if is_exact:
        return map_Z_meas_to_bit(job.result().values)
    else:
        return job.result()


def map_Z_meas_to_bit(est_res):
    bin_res = []
    for i in range(len(est_res)):
        if est_res[i] == 1:
            bin_res.append(0)
        if est_res[i] == -1:
            bin_res.append(1)
    return bin_res

#### Visualization

In [12]:
import matplotlib.pyplot as plt

def draw_graph(labels, values):
    plt.plot(labels, values, '-o')
    plt.xlabel('Observables')
    plt.ylabel('Expectation Value')
    plt.show()

### Circuit Names

In [104]:
def name_adder(n: int):
    return str(n) + '_Add'

def name_mod_adder(n: int):
    return 'Add mod 2^' + str(n)

def name_add_with_0(n: int):
    return str(n) + '_Copy'

def name_shift(n: int, k: int):
    return str(n) + '_Shift to ' + str(k)

def name_out_of_place_mod_mul(n: int, a: int):
    return 'x' + str(a) + ' mod 2^' + str(n) + ' (OP)'

def name_in_place_mod_mul(n: int, a: int):
    return 'x' + str(a) + ' mod 2^' + str(n) + ' (IP)'

def name_mod_exp(n: int, a: int):
    return str(a) + '^x mod 2^' + str(n)

def name_period_finder(n: int, a: int):
    return 'Period Finder of ' + str(a) + '^x mod 2^' + str(n)

def name_qasm_file(N: int, a: int, eps: int, op):
    return 'Shor_N' + str(N) + '_a' + str(a) + '_eps' + str(eps) + '_op' + str(op) + '.qasm'

## Compilation

### Gate Sets

In [35]:
from enum import Enum

class GateSet(Enum):
    Pauli = ['x', 'y', 'z']
    Clifford = Pauli + ['h', 's', 'sdg', 'cx']
    Clifford_T = Clifford + ['t', 'tdg']
    Clifford_T_1q_rotations = Clifford_T + ['u']

### Compile to a Gate Set

In [36]:
from qiskit import transpile

def compile(qc: QuantumCircuit, gate_set: GateSet, optimization_level):
    compiled_qc = transpile(qc, basis_gates=gate_set.value, optimization_level=optimization_level)
    return compiled_qc

### Compile to OpenQASM

In [37]:
from qiskit import qasm2

def to_qasm(qc: QuantumCircuit, file_name):
    qasm2.dump(qc, file_name)


def show_qasm(qc: QuantumCircuit):
    openqasm_code = qasm2.dumps(qc)
    print(openqasm_code)
    return openqasm_code

## Binary Integer Addition

### Cuccaro Adder

In [15]:
# Returns circuit of cuccaro adder for 'n' bit inputs, using (2n + 2) qubits
# 1 ancilla for carry-in (initially 0)
# n qubits for first input
# n qubits for second input which will be rewritten by sum
# 1 qubit for most-significant bit of sum
# |0> |a> |b> |z> --> |0> |a> |a + b> |(a + b)_n>
def cuccaro_adder(n: int):
    A = QuantumRegister(n, name='A')
    B = QuantumRegister(n, name='B')
    Z = QuantumRegister(1, name='Z')
    X = QuantumRegister(1, name='X')

    qc = QuantumCircuit(X, A, B, Z, name=name_adder(n))
    
    # small n
    if n < 4:
        return small_adder(n, qc, A, B, X, Z)

    for i in range(1, n):
        qc.cx(A[i], B[i])

    qc.cx(A[1], X)

    qc.ccx(A[0], B[0], X) #TODO: Toffoli to Clifford + T
    qc.cx(A[2], A[1])

    qc.ccx(X, B[1], A[1]) #TODO: Toffoli to Clifford + T
    qc.cx(A[3], A[2])

    for i in range(2, n-2):
        qc.ccx(A[i - 1], B[i], A[i]) #TODO: Toffoli to Clifford + T
        qc.cx(A[i + 2], A[i + 1])

    qc.ccx(A[n - 3], B[n - 2], A[n - 2]) #TODO: Toffoli to Clifford + T
    qc.cx(A[n - 1], Z)

    qc.ccx(A[n - 2], B[n - 1], Z) #TODO: Toffoli to Clifford + T
    for i in range(1, n - 1):
        qc.x(B[i])

    qc.cx(X, B[1])
    for i in range(2, n):
        qc.cx(A[i - 1], B[i])
    
    qc.ccx(A[n - 3], B[n - 2], A[n - 2]) #TODO: Toffoli to Clifford + T

    for i in range(n - 3, 1, -1):
        qc.ccx(A[i - 1], B[i], A[i]) #TODO: Toffoli to Clifford + T
        qc.cx(A[i + 2], A[i + 1])
        qc.x(B[i + 1])

    qc.ccx(X, B[1], A[1]) #TODO: Toffoli to Clifford + T
    qc.cx(A[3], A[2])
    qc.x(B[2])

    qc.ccx(A[0], B[0], X) #TODO: Toffoli to Clifford + T
    qc.cx(A[2], A[1])
    qc.x(B[1])

    qc.cx(A[1], X)

    for i in range(n):
        qc.cx(A[i], B[i])
    
    return qc


# Precondition: 0 < n < 4
def small_adder(n: int, qc: QuantumCircuit, A: QuantumRegister, B: QuantumRegister, X: QuantumRegister, Z: QuantumRegister):
    if n == 3:
        return three_bit_adder(qc, A, B, X, Z)
    if n == 2:
        return two_bit_adder(qc, A, B, X, Z)
    if n == 1:
        return one_bit_adder(qc, A, B, Z)
    return qc


def one_bit_adder(qc, A, B, Z):
    qc.ccx(A[0], B[0], Z)
    qc.cx(A[0], B[0])
    return qc


def two_bit_adder(qc, A, B, X, Z):
    qc.ccx(A[0], B[0], X)
    qc.ccx(A[1], B[1], Z)
    qc.cx(A[1], B[1])
    qc.ccx(X, B[1], Z)
    qc.cx(X, B[1])
    qc.ccx(A[0], B[0], X)
    qc.cx(A[0], B[0])
    return qc


def three_bit_adder(qc, A, B, X, Z):
    qc.ccx(A[0], B[0], X)

    for i in range(1, 3):
        qc.cx(A[i], B[i])
    
    qc.cx(A[1], X)
    qc.ccx(X, B[1], A[1])

    qc.cx(A[2], A[1])
    qc.ccx(A[1], B[2], A[2])

    qc.cx(A[2], Z)

    qc.ccx(A[1], B[2], A[2])
    qc.cx(A[2], A[1])
    qc.cx(A[1], B[2])

    qc.ccx(X, B[1], A[1])
    qc.cx(A[1], X)
    qc.cx(X, B[1])

    qc.ccx(A[0], B[0], X)
    qc.cx(A[0], B[0])

    return qc


def _test_cuccaro_adder():
    qc = cuccaro_adder(3)
    print(qc)
    # qc.draw(output='mpl')

### Modular Adder

In [16]:
# Returns circuit of cuccaro adder for 'n' bit inputs mod 2^n, using (2n + 1) qubits
# 1 ancilla for carry-in (initially 0)
# n qubits for first input
# n qubits for second input which will be rewritten by sum
# |0> |a> |b> --> |0> |a> |(a + b) mod 2^n>
def modular_adder(n: int):
    X = QuantumRegister(1, name='X')
    A = QuantumRegister(n, name='A')
    B = QuantumRegister(n, name='B')

    qc = QuantumCircuit(X, A, B, name=name_mod_adder(n))

    # very small n
    if n == 1:
        qc.cx(A[0], B[0])
        return qc

    qc_1 = cuccaro_adder(n - 1).to_gate()

    qc = qc.compose(qc_1, qubits=qubit_map_cuccaro_to_modular(n))

    qc.cx(A[n - 1], B[n - 1])

    return qc


def qubit_map_cuccaro_to_modular(n):
    map = [0]

    for i in range(1, n):
        map.append(i)

    for i in range(n + 1, 2 * n + 1):
        map.append(i)

    return map


def _test_modular_adder():
    qc = modular_adder(3)
    print(qc)
    # qc.draw(output='mpl')

### Add with 0 (Copy)

In [17]:
# Returns circuit of adder for 'n' bit inputs where the second input is 0 using (2n) qubits
# i.e. copis 'n' bits to another 'n' 0 bits
# n qubits for first input
# n qubits for second input which will be rewritten by sum
# |a> |0> --> |a> |a>
def adder_with_0(n: int):
    A = QuantumRegister(n, name='A')
    X = QuantumRegister(n, name='X')

    qc = QuantumCircuit(A, X, name=name_add_with_0(n))

    for i in range(n):
        qc.cx(A[i], X[i])

    return qc


def _tet_adder_with_0():
    qc = adder_with_0(7)
    print(qc)
    # qc.draw(output='mpl')

### Adder Test

#### Adder Initialization

In [ ]:
def _add(n, a, b):
    qc_add = cuccaro_adder(n)

    qc = QuantumCircuit(*qc_add.qregs)

    qc = init_qreg_val(n, a, qc, [i for i in range(1, n + 1)])
    qc = init_qreg_val(n, b, qc, [i for i in range(n + 1, 2 * n + 1)])

    qc = qc.compose(qc_add)

    return qc


def _mod_add(n, a, b):
    qc_add = modular_adder(n)

    qc = QuantumCircuit(*qc_add.qregs)

    qc = init_qreg_val(n, a, qc, [i for i in range(1, n + 1)])
    qc = init_qreg_val(n, b, qc, [i for i in range(n + 1, 2 * n + 1)])

    qc = qc.compose(qc_add)

    return qc

#### Measurement

In [19]:
def test_mod_add(n, a, b):
    qc = _mod_add(n, a, b)

    res = test_all(qc, is_exact=True)
    labels = labels_mod_add(n)

    draw_graph(labels, res)
    return qc


def test_mod_add_dag(n, a, b):
    qc = _mod_add(n, a, b)

    qc_1 = modular_adder(n)
    qc_1 = qc_1.inverse()

    qc = qc.compose(qc_1)

    res = test_all(qc, is_exact=True)
    labels = labels_mod_add(n)

    draw_graph(labels, res)
    return qc


def labels_mod_add(n):
    labels = ['X']
    for i in range(n):
        labels.append('A' + str(i))
    for i in range(n):
        labels.append('B' + str(i))
    return labels


def _test_mod_add():
    # qc = test_mod_adder(n, a, b)
    qc = test_mod_add_dag(n=3, a=1, b=2)
    print(qc)
    # qc.draw('mpl')

## Modular Multiplication

### Bit Shift

In [20]:
# Returns circuit for shifting an 'n'-bit string, 'k' bits to the left, i.e. *= 2^k,
# using 'k' 0 ancillas which will be dirty by storing the 'k' most-significant bits at the end,
# and (2n) CNOTs
# Precondition: k < n
# |a> |0>^k --> |0>^k |a>
def left_shifter(n: int, k: int):
    X = QuantumRegister(n, name='X')
    A = QuantumRegister(k, name='A')
    
    qc = QuantumCircuit(X, A, name=name_shift(n, k))

    for i in range(n - 1, -1, -1):
        qc.cx(i, i + k)
        qc.cx(i + k, i)
    
    return qc


def _test_left_shifter():
    qc = left_shifter(n=7, k=3)
    print(qc)
    # qc.draw(output='mpl')

### Out of Place Modular Multiplier

In [22]:
# Returns circuit for out-of-place modular 2^n multiplication of an 'n' bit input with 'a' , using (3n) qubits
# n qubits for input
# n qubits initially 0 to be rewritten by mul result
# n - 1 ancillas initially 0 to become dirty with shifts 
# 1 ancilla initially 0 for carry-in to be used by adders
# |b> |0>^n |0>^n |--> |b> |b.a mod 2^n> |garabage>^(n - 1) |0>
def out_of_place_modular_multiplier(n: int, a: int):
    B = QuantumRegister(n, name='B')
    M = QuantumRegister(n, name='M')
    A = QuantumRegister(n - 1, name='G')
    X = QuantumRegister(1, name='X')

    qc = QuantumCircuit(B, M, A, X, name=name_out_of_place_mod_mul(n, a))

    bin_a = n_bit_binary(n, a)
    last_shift = -1

    for i in range(n - 1, -1, -1):
        if bin_a[i] == '1':

            if last_shift == -1: # first on bit of 'a'
                qc = add_with_0_for_mod_mul(n, n, qc)
            
            else:
                k = last_shift - i

                # M << k
                qc = shift_left_for_mod_mul(n, k, last_shift, qc)

                # M += B
                qc = add_with_0_for_mod_mul(n, k, qc)
                qc = mod_add_for_mod_mul(n, k, qc)

            last_shift = i
            
        elif i == 0: # shift to adjust if last bit of 'a' was off, useless since GCD(a, 2^n) = 1
            k = last_shift - i
            qc = shift_left_for_mod_mul(n, k, last_shift, qc)
        
    return qc


def shift_left_for_mod_mul(n: int, k: int, last_shift: int, qc: QuantumCircuit):
    qc_shift = left_shifter(n, k).to_gate()

    map = [i for i in range(n, 2 * n)]
    map += [shift_garbage(i, n, last_shift) for i in range(k)]
    
    return qc.compose(qc_shift, qubits=map)


def shift_garbage(i: int, n: int, last_shift: int):
    return n - 1 - last_shift + i + 2 * n


def add_with_0_for_mod_mul(n: int, k: int, qc: QuantumCircuit):
    qc_add_0 = adder_with_0(k).to_gate()

    map = [i for i in range(k)]
    map += [i + n for i in range(k)]

    return qc.compose(qc_add_0, qubits=map)


def mod_add_for_mod_mul(n: int, k: int, qc: QuantumCircuit):
    qc_add_1 = modular_adder(n - k).to_gate()

    map = [3 * n - 1]
    map += [i for i in range(k, n)]
    map += [i + n for i in range(k, n)]
    
    return qc.compose(qc_add_1, qubits=map)

#### Out of Place Modular Multiplier Test

In [23]:
def _op_mod_mul(a, b, n):
    qc_mul = out_of_place_modular_multiplier(n, a)

    qc = QuantumCircuit(*qc_mul.qregs)
    qc = init_qreg_val(n, b, qc, map=[i for i in range(n)])
    qc = qc.compose(qc_mul)

    return qc


def test_op_mod_mul(a, b, n):
    qc = _op_mod_mul(a, b, n)

    res = test_all(qc, is_exact=True)
    labels = labels_op_mod_mul(n)

    draw_graph(labels, res)
    return qc


def labels_op_mod_mul(n):
    labels = []
    for i in range(n):
        labels.append('B' + str(i))
    for i in range(n):
        labels.append('M' + str(i))
    for i in range(n - 1):
        labels.append('A' + str(i))
    labels.append('X')
    return labels


def _test_out_of_place_modular_multiplier():
    # a = 5 #0101
    # b = 5 #0101
    # a * b mod 2^n = 9 = 1001
    # A = 010
    qc = test_op_mod_mul(a=5, b=5, n=4)
    # qc.draw(output='mpl')

### In-place Modular Multiplier

In [24]:
# Returns circuit for in-place modular 2^n multiplication of an 'n' bit input with 'a' , using (4n) qubits
# n qubits for input
#     2n qubits initially 0 to be rewritten by mul result
#     n - 1 ancillas initially 0 to become dirty with shifts 
#     1 ancilla initially 0 for carry-in to be used by adders
# |b> |0>^n |0>^n |0>^n |--> |0>^n |0>^n |0>^n |b.a mod 2^n>
def in_place_modular_multiplier(n: int, a: int):
    
    a_inverse = multiplicative_inverse(a, 2**n)
    if a_inverse == -1:
        return
    
    B = QuantumRegister(n, name='B')
    M = QuantumRegister(n, name='M')
    A = QuantumRegister(n - 1, name='A')
    X = QuantumRegister(1, name='X')
    C = QuantumRegister(n, name='C')

    qc = QuantumCircuit(B, M, A, X, C, name=name_in_place_mod_mul(n, a))

    B_map = [i for i in range(n)]
    M_map = [i + n for i in range(n)]
    A_map = [i + 2 * n for i in range(n)]
    C_map = [i + 3 * n for i in range(n)]

    qc_U_a = out_of_place_modular_multiplier(n, a).to_gate()
    qc_copy = adder_with_0(n).to_gate()
    qc_U_a_dag = qc_U_a.inverse()
    qc_U_a_inv = out_of_place_modular_multiplier(n, a_inverse).to_gate()
    qc_U_a_inv_dag = qc_U_a_inv.inverse()
    
    qc = (qc.compose(qc_U_a, qubits=B_map + M_map + A_map)       #[B, M, A, C] = [b, ab mod N, g(a, b), 0]
          .compose(qc_copy, qubits=M_map + C_map)                #[B, M, A, C] = [b, ab mod N, g(a, b), ab mod N]
          .compose(qc_U_a_dag, qubits=B_map + M_map + A_map)     #[B, M, A, C] = [b, 0, 0, ab mod N]
          .compose(qc_U_a_inv, qubits=C_map + M_map + A_map)     #[B, M, A, C] = [b, b, g(a^-1, ab), ab mod N]
          .compose(qc_copy, qubits=B_map + M_map)                #[B, M, A, C] = [b, 0, g(a^-1, ab), ab mod N]
          .compose(qc_U_a_inv_dag, qubits=C_map + B_map + A_map) #[B, M, A, C] = [0, 0, 0, ab mod N]
          .compose(qc_copy, qubits=C_map + B_map)                #[B, M, A, C] = [ab mod N, 0, 0, ab mod N]
          .compose(qc_copy, qubits=B_map + C_map))               #[B, M, A, C] = [ab mod N, 0, 0, 0]
    return qc

#### In-Place Modular Multiplier Test

In [25]:
def _ip_mod_mul(a, b, n):
    qc_mul = in_place_modular_multiplier(n, a)

    qc = QuantumCircuit(*qc_mul.qregs)
    qc = init_qreg_val(n, b, qc, map=[i for i in range(n)])
    qc = qc.compose(qc_mul)

    return qc


def test_ip_mod_mul(a, b, n):
    qc = _ip_mod_mul(a, b, n)

    res = test_all(qc, is_exact=True)
    labels = labels_ip_mod_mul(n)

    draw_graph(labels, res)
    return qc


def labels_ip_mod_mul(n):
    labels = []
    for i in range(n):
        labels.append('B' + str(i))
    for i in range(n):
        labels.append('M' + str(i))
    for i in range(n - 1):
        labels.append('A' + str(i))
    labels.append('X')
    for i in range(n):
        labels.append('C' + str(i))
    return labels


def _test_in_place_modular_multiplier():
    #ab mod 16 = 9 = 1001
    qc = test_ip_mod_mul(a=5, b=5, n=4)
    # qc.draw(output='mpl')

### Controlled Modular Multiplier

In [26]:
def controlled_modular_multiplier(n: int, a:int):
    U_a = in_place_modular_multiplier(n, a)
    cU_a = U_a.to_gate().control(1)

    qreg = QuantumRegister(len(U_a.qubits) + 1)
    qc = QuantumCircuit(qreg, name=name_in_place_mod_mul(n, a))
    qc.append(cU_a, qreg)

    return qc

## Modular Exponentiation

In [27]:
# Returns circuit for exponentiation of an int 'a' to the 'n' bit input 'x' mod 2^n, using (5n) qubits
# n qubits for exponent input
# n qubits initially 0..01 to be rewritten by exp result
# 3n ancillas initially 0
# |x> |1> |0> --> |x> |a^x mod 2^n> |0>
def modular_exponent(n: int, a: int):
    X = QuantumRegister(n, name='X')
    B = QuantumRegister(n, name='B')
    A = QuantumRegister(2 * n, name='A')
    M = QuantumRegister(n, name='M')

    qc = QuantumCircuit(X, B, A, M, name=name_mod_exp(n, a))

    B_map = [j + n for j in range(n)]
    A_map = [j + 2 * n for j in range(2 * n)]
    M_map = [j + 4 * n for j in range(n)]

    for i in range(n):
        a_2i = a ** (2 ** i)
        qc_2i = controlled_modular_multiplier(n, a_2i)
        
        qc = qc.compose(qc_2i, qubits=[i] + B_map + A_map + M_map)
    
    return qc

### Modular Exponentiation Test

In [32]:
def _mod_exp(a, x, n):
    qc_exp = modular_exponent(n, a)

    qc = QuantumCircuit(*qc_exp.qregs)
    qc = init_qreg_val(n, x, qc, map=[i for i in range(n)])
    qc = init_qreg_val(n, 1, qc, map=[i + n for i in range(n)])
    qc = qc.compose(qc_exp)

    return qc


def test_mod_exp(a, x, n):
    qc = _mod_exp(a, x, n)

    res = test_all(qc, is_exact=True)
    labels = labels_for_test_mod_exp(n)

    draw_graph(labels, res)
    return qc


def labels_for_test_mod_exp(n):
    labels = []
    for i in range(n):
        labels.append('X' + str(i))
    for i in range(n):
        labels.append('B' + str(i))
    for i in range(2 * n):
        labels.append('A' + str(i))
    for i in range(n):
        labels.append('M' + str(i))
    return labels


def _test_modular_exponent():
    #3^4 mod 16 = 81 = 1
    qc = test_mod_exp(a=3, x=6, n=3)
    # qc.draw(output='mpl')

## Period Finding

In [88]:
from qiskit.circuit.library import QFT
from qiskit import ClassicalRegister

def period_finder(n: int, a: int, eps: int, decompose=False, optimization_level=1):
    X = QuantumRegister(n, name='X')
    B = QuantumRegister(n, name='B')
    A = QuantumRegister(3 * n, name='A')
    
    C = ClassicalRegister(n, name='C')

    qc = QuantumCircuit(X, B, A, C, name=name_period_finder(n, a))

    X_map = [i for i in range(n)]

    qc_init = uniform_superposiotion_creator(n)
    qc_exp = modular_exponent(n, a)
    qc_qft = QFT(n, eps, do_swaps=False, inverse=True)

    if decompose:
        qc_exp = compile(qc_exp, GateSet.Clifford_T_1q_rotations, optimization_level)
        qc_qft = compile(qc_qft, GateSet.Clifford_T_1q_rotations, optimization_level)

    qc = (qc.compose(qc_init, X_map)
          .compose(qc_exp)
          .compose(qc_qft, qubits=X_map))
    
    qc.measure(X, C)

    return qc


### Period Finding Test

In [93]:
def _find_period(a, n, eps, decompose, optimization_level=1):
    qc_pf = period_finder(n, a, eps, decompose, optimization_level)

    qc = QuantumCircuit(*qc_pf.qregs)
    qc = init_qreg_val(n, 1, qc, map=[i + n for i in range(n)])
    qc = qc.compose(qc_pf)

    return qc


def test_pf(a, n):
    qc = _find_period(a, n, eps=0, decompose=False)

    res = test_all(qc, is_exact=False)
    labels = labels_for_test_mod_exp(n)

    draw_graph(labels, res)
    return qc


def labels_for_test_pf(n):
    labels = []
    for i in range(n):
        labels.append('X' + str(i))
    for i in range(n):
        labels.append('B' + str(i))
    for i in range(2 * n):
        labels.append('A' + str(i))
    for i in range(n):
        labels.append('M' + str(i))
    return labels


def _test_period_finder():
    #3^4 mod 16 = 81 = 1
    qc = test_pf(a=3, n=4)
    # qc.draw(output='mpl')


## Inspection

In [72]:
# The circuit should be compiled before inspection for more accurate results
def count(gates, qc: QuantumCircuit):
    c = 0
    for gate, _, _ in qc:
        if gate.name in gates:
            c += 1
    return c


def t_count(qc: QuantumCircuit):
    gates = ['t', 'tdg']
    return count(gates, qc)


def u_count(qc: QuantumCircuit):
    return count(['u'], qc)


def qubit_count(qc: QuantumCircuit):
    return len(qc.qubits)

### Single-qubit Rotation Approximation

In [86]:
import math

def pure_t_count(qc: QuantumCircuit, eps: int):
    t = t_count(qc)
    u = u_count(qc)
    tu = int(t_count_for_rotations(u, eps))
    return t + tu


def t_count_for_rotations(u: int, eps: int):
    return 3 * math.log(10 ** eps)

## Interface

In [105]:
def statistics(a, N, eps, optimization_level):
    n = log_2(N)

    qc = _find_period(a, n, eps, decompose=True, optimization_level=optimization_level)

    t = t_count(qc)
    u = u_count(qc)
    tt = pure_t_count(qc, eps)
    q = qubit_count(qc)

    print('Optimization level: ', optimization_level)

    print('T gates: ', t)
    print('Single-qubit rotations: ', u)
    print('Total T count: ', tt)
    print('Qubits: ', q)

    print('-------------------------')

    to_qasm(qc, file_name=name_qasm_file(n, a, eps, optimization_level))
    # show_qasm(qc)

    # qc.draw('mpl')
    # print(qc)

    return qc


a = 3
N = 32
eps = 7

statistics(a, N, eps, 0)
statistics(a, N, eps, 1)

/var/folders/81/cnr4jdbx62qcrhmskj4dn8y00000gn/T/ipykernel_33358/760517074.py:4: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for gate, _, _ in qc:


Optimization level:  0
T gates:  5404
Single-qubit rotations:  1488
Total T count:  5452
Qubits:  25
-------------------------
Optimization level:  1
T gates:  4632
Single-qubit rotations:  2098
Total T count:  4680
Qubits:  25
-------------------------


/var/folders/81/cnr4jdbx62qcrhmskj4dn8y00000gn/T/ipykernel_33358/760517074.py:4: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for gate, _, _ in qc:
